In [1]:
from SmartAnno.utils.ReviewRBInit import ReviewRBInit
from SmartAnno.utils.ReviewRBLoop import ReviewRBLoop
from SmartAnno.utils.ReviewMLInit import ReviewMLInit
from SmartAnno.utils.ReviewMLLoop import ReviewMLLoop
from SmartAnno.gui.Workflow import Workflow
from sqlalchemy_dao import Dao
from SmartAnno.db.ORMs import Document
from SmartAnno.utils.IntroStep import IntroStep
from SmartAnno.gui.PreviousNextWidgets import PreviousNextHTML
import sqlalchemy_dao
import os
from  conf.ConfigReader import ConfigReader
from SmartAnno.utils.KeywordsEmbeddingExtender import KeywordsEmbeddingExtender
from SmartAnno.utils.KeywordsEmbeddingExtenderSetup import KeywordsEmbeddingExtenderSetup
from SmartAnno.utils.TreeSet import TreeSet
import logging
logging.getLogger().setLevel(logging.DEBUG)
ConfigReader('../conf/smartanno_conf.json')

from SmartAnno.models.GloveModel import GloveModel
from  conf.ConfigReader import ConfigReader
from threading import Thread

def prepareGloveModel():
    ConfigReader('../conf/smartanno_conf.json')
    glove_path = ConfigReader.getValue('glove/model_path')
    glove_vocab = ConfigReader.getValue('glove/vocab')
    glove_vector = ConfigReader.getValue('glove/vector')
    GloveModel(word2vec_file=glove_path, vocab=glove_vocab, vect=glove_vector)
    gm=GloveModel.glove_model
    

thread_gm = Thread(target=prepareGloveModel)
thread_gm.start()    



wf=Workflow()
wf.filters={'TypeA':TreeSet(['English']),'TypeB':TreeSet(['Spanish'])}
wf.append(KeywordsEmbeddingExtenderSetup(name='w_e_extender_setup'))
wf.append(KeywordsEmbeddingExtender(name='w_e_extender', max_query=40))
wf.append(KeywordsEmbeddingExtenderSetup(name='w_e_extender_setup'))
wf.start()

ToggleButtonsMultiSelectionInBox(children=(VBox(layout=Layout(display='flex')),), _dom_classes=('btn-group',))

Load glove model in the backend...


VBox(children=(HTML(value='<h4>Extend keywords through Word Embedding</h4><p>Please select which keywords you want to check the synonyms from the word embedding (currently only single word works for the word embedding model):'), Label(value='TypeA:'), ToggleButtonsMultiSelection(children=(ToggleButton(value=False, description='English', layout=Layout(margin='2')),), _dom_classes=('btn-group',)), Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), Label(value='TypeB:'), ToggleButtonsMultiSelection(children=(ToggleButton(value=False, description='Spanish', layout=Layout(margin='2')),), _dom_classes=('btn-group',)), Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='Previous', layout=Layout(width='90px'), style=ButtonStyle(button_color='SILVER')), Button(description='Next', layout=Layout(left='100px', width='90px'), style=ButtonStyle(button_color='SANDYBROWN'))), layout=Layout(left='10%', width='80%'))), layout=Layout(display='flex'))

In [11]:
wf.filters

{'TypeA': ['English', 'language', 'languages', 'speaking'],
 'TypeB': ['Spanish', 'catalan', 'dutch', 'french', 'german', 'italian', 'mexican', 'portuguese', 'spain']}

In [ ]:
type(extending[0])

In [1]:
import logging

import sqlalchemy_dao
from sqlalchemy_dao import Dao

from SmartAnno.utils.ConfigReader import ConfigReader
from SmartAnno.db.ORMs import Filter
from SmartAnno.gui.Workflow import Workflow
from SmartAnno.utils.AnnotationTypeDef import AnnotationTypeDef
from SmartAnno.utils.IntroStep import IntroStep
from SmartAnno.utils.KeywordsFiltering import KeywordsFiltering
from SmartAnno.utils.KeywordsEmbeddingExtender import KeywordsEmbeddingExtender
from SmartAnno.utils.KeywordsEmbeddingExtenderSetup import KeywordsEmbeddingExtenderSetup

logging.getLogger().setLevel(logging.DEBUG)

ConfigReader('../conf/smartanno_conf.json')

from SmartAnno.models.GloveModel import GloveModel
from threading import Thread

def prepareGloveModel():
    ConfigReader('../conf/smartanno_conf.json')
    glove_path = ConfigReader.getValue('glove/model_path')
    glove_vocab = ConfigReader.getValue('glove/vocab')
    glove_vector = ConfigReader.getValue('glove/vector')
    GloveModel(word2vec_file=glove_path, vocab=glove_vocab, vect=glove_vector)
    gm=GloveModel.glove_model
    

thread_gm = Thread(target=prepareGloveModel)
thread_gm.start()  

wf = Workflow(config_file=ConfigReader.config_file)
wf.api_key = ConfigReader.getValue("api_key")
wf.dao = Dao('sqlite+pysqlite:///../data/test.sqlite', sqlalchemy_dao.POOL_DISABLED)
wf.task_name = 'language'
wf.append(AnnotationTypeDef(
    '<h3>Annotation types:</h3><p>List all the types you want to identify below. Each type per line.<br/>If you'
    'have too many types, try set up them separately, so that you won&apos;t need to choose from a long list '
    'for each sample. </p>', name='types'))
wf.append(KeywordsFiltering(
    name='keywords'))
wf.append(KeywordsEmbeddingExtenderSetup(name='w_e_extender_setup'))
wf.append(KeywordsEmbeddingExtender(name='w_e_extender', max_query=40))

wf.start()

wf.steps[0].complete()

with wf.dao.create_session() as session:
    records = session.query(Filter).filter(Filter.task_id == wf.task_id) \
        .filter(Filter.type_name == 'Eng')
    record = records.first()
    record.keyword = 'Eng\nEnglish'
wf.steps[1].complete()    

VBox(children=(HTML(value='<h4>Extend keywords through Word Embedding</h4><p>Please select which keywords you want to check the synonyms from the word embedding (currently only single word works for the word embedding model):'), Label(value='Eng:'), ToggleButtonsMultiSelectionInBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, description='Eng', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='Eng'), ToggleButton(value=False, description='English', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='English'))),), layout=Layout(display='flex')),), _dom_classes=('btn-group',)), Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), Label(value='PossibleEng:'), ToggleButtonsMultiSelectionInBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, description='administrator', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='administrator'), ToggleButton(value=False, description='assistant', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='assistant'), ToggleButton(value=False, description='consultant', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='consultant'), ToggleButton(value=False, description='coordinator', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='coordinator'), ToggleButton(value=False, description='designer', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='designer'), ToggleButton(value=False, description='designers', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='designers'))), HBox(children=(ToggleButton(value=False, description='director', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='director'), ToggleButton(value=False, description='executive', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='executive'), ToggleButton(value=False, description='fluency', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='fluency'), ToggleButton(value=False, description='fluent', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='fluent'), ToggleButton(value=False, description='fluently', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='fluently'), ToggleButton(value=False, description='manager', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='manager'))), HBox(children=(ToggleButton(value=False, description='managers', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='managers'), ToggleButton(value=False, description='proficient', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='proficient'), ToggleButton(value=False, description='supervisor', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='supervisor')))), layout=Layout(display='flex')),), _dom_classes=('btn-group',)), Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), Label(value='NotEng:'), ToggleButtonsMultiSelectionInBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, description='Chinese', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='Chinese'), ToggleButton(value=False, description='German', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='German'), ToggleButton(value=False, description='Italian', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='Italian'), ToggleButton(value=False, description='Japanese', layout=Layout(margin='1', min_width='160px', width='100%'), tooltip='Japanese'), ToggleButton(value=False, description='Korean', layout=Layout(margin='1', min_width='160px', width='100%'), toolti